In [58]:
import keras
from keras import layers


In [59]:
import keras_tuner


In [60]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=[28, 28]))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=1),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

In [61]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="C:/Users/nisse/OneDrive - Högskolan Väst/IAI600/Lab 7",
    project_name="helloworld",
)

In [62]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 1, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [63]:
import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [64]:
import tensorflow

stop_early = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [65]:
tuner.search(x_train, y_train, epochs=50, batch_size=784, validation_data=(x_val, y_val), callbacks=[stop_early])

Trial 3 Complete [00h 01m 09s]
val_accuracy: 0.9781500101089478

Best val_accuracy So Far: 0.98335000872612
Total elapsed time: 00h 02m 36s
INFO:tensorflow:Oracle triggered exit


In [66]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 504)               395640    
                                                                 
 dropout (Dropout)           (None, 504)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5050      
                                                                 
Total params: 400,690
Trainable params: 400,690
Non-trainable params: 0
_________________________________________________________________


In [67]:
tuner.results_summary()

Results summary
Results in C:/Users/nisse/OneDrive - Högskolan Väst/IAI600/Lab 7\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 1
units_0: 504
activation: relu
dropout: True
lr: 0.004163154630890509
units_1: 129
units_2: 285
units_3: 126
units_4: 188
Score: 0.98335000872612

Trial 2 summary
Hyperparameters:
num_layers: 4
units_0: 336
activation: relu
dropout: False
lr: 0.000254400127046171
units_1: 510
units_2: 105
units_3: 445
units_4: 99
Score: 0.9781500101089478

Trial 0 summary
Hyperparameters:
num_layers: 5
units_0: 101
activation: tanh
dropout: False
lr: 0.00011894173793507069
units_1: 32
units_2: 32
units_3: 32
units_4: 32
Score: 0.9683500230312347


In [68]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp.values)
model = tuner.hypermodel.build(best_hp)

{'num_layers': 1, 'units_0': 504, 'activation': 'relu', 'dropout': True, 'lr': 0.004163154630890509, 'units_1': 129, 'units_2': 285, 'units_3': 126, 'units_4': 188}


In [69]:
x_allt = np.concatenate((x_train, x_val), axis=0)
y_allt = np.concatenate((y_train, y_val), axis=0)

In [70]:
lastmodel = model.fit(x_allt, y_allt, epochs=50, batch_size=784, callbacks=[stop_early])

Epoch 1/50
77/77 [==============================] - 2s 15ms/step - loss: 0.3489 - accuracy: 0.8932
Epoch 2/50
77/77 [==============================] - 1s 16ms/step - loss: 0.1328 - accuracy: 0.9604
Epoch 3/50
77/77 [==============================] - 1s 15ms/step - loss: 0.0930 - accuracy: 0.9717
Epoch 4/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0717 - accuracy: 0.9781
Epoch 5/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0556 - accuracy: 0.9833
Epoch 6/50
77/77 [==============================] - 1s 15ms/step - loss: 0.0453 - accuracy: 0.9859
Epoch 7/50
77/77 [==============================] - 1s 15ms/step - loss: 0.0377 - accuracy: 0.9881
Epoch 8/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0326 - accuracy: 0.9898
Epoch 9/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0279 - accuracy: 0.9913
Epoch 10/50
77/77 [==============================] - 1s 14ms/step - loss: 0.0253 - accuracy: 0.9917
Epoch 11/

In [71]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.1017 - accuracy: 0.9832
Test accuracy: 0.9832000136375427


In [72]:
# https://www.tensorflow.org/tutorials/keras/keras_tuner
# https://keras.io/keras_tuner/getting_started/
# https://campus.datacamp.com/courses/introduction-to-deep-learning-with-keras/improving-your-model-performance?ex=9